In [ ]:
from tqdm import tqdm
from time import time
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from pyspark.context import SparkContext,SparkConf

from Functions import ox,px,cx,rsm,twors,im,cim,psm
from Job import Job
from Population import Population

# init spark context
conf = SparkConf()
sc = SparkContext(conf=conf)

sc.addPyFile('Functions.py')
sc.addPyFile('Population.py')
sc.addPyFile('Job.py')

# load cities data from file
DIST_MAT = np.loadtxt('att48_d.txt')
OPTIMAL = 33523
repeats = 20

# 7. Operators Comparison

In [ ]:
# ga paremeters
POP_SELECT= .2
POP_CROSS = .9
POP_MUT = .1
POP_ELITE =.1

pop_size = 1000
target_generation = 1000


# function to be compared
crossover_funcs = [ox,px,cx]
c_names =['OX','PX','CX']
mutation_funcs= [rsm,twors,im,cim,psm]


# run and plot 
plt.figure(figsize=(28,8))
for i,cross in enumerate(crossover_funcs):   
    plt.subplot(1,3,i+1)
    for mut in mutation_funcs:
        jobs =[Job(pop_size,
               POP_SELECT,
               POP_CROSS,
               POP_MUT,
               POP_ELITE,
               cross,
               mut,
               DIST_MAT) for _ in range(repeats)]
        
        jobs_RDD = sc.parallelize(jobs)
        jobs_RDD = jobs_RDD.map(lambda j: j.generate_population())
        jobs_RDD = jobs_RDD.map(lambda j: j.run(target_generation))
        jobs_res = jobs_RDD.map(lambda j: j.get_history()).collect()
        
        jobs_res = np.array(jobs_res).mean(axis=0)
        plt.plot(jobs_res)
        
    plt.title(f'{c_names[i]} Crossover')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.legend(['rsm', 'twors','im','cim','psm'], title='Mutation', loc='lower left')

plt.show()

# 8.1 Population

In [ ]:
# ga paremeters
POP_SELECT= .2
POP_CROSS = .9
POP_MUT = .1
POP_ELITE =.1

# function to be compared
crossover_funcs = [cx]
c_names =['CX']
mutation_funcs= [rsm]

# run parameters
pop_size = 50
target_generation = 1000


# run and plot 
for w in range(20):
    for i,cross in enumerate(crossover_funcs):   
        for mut in mutation_funcs:
            # Parallel
            jobs =[Job(pop_size,
                   POP_SELECT,
                   POP_CROSS,
                   POP_MUT,
                   POP_ELITE,
                   cross,
                   mut,
                   DIST_MAT) for _ in range(repeats)]

            jobs_RDD = sc.parallelize(jobs)
            jobs_RDD = jobs_RDD.map(lambda j: j.generate_population())
            jobs_RDD = jobs_RDD.map(lambda j: j.run(target_generation))
            jobs_res = jobs_RDD.map(lambda j: j.get_history()).collect()
            jobs_res = np.array(jobs_res).mean(axis=0)

            
    pop_size+=(w*5)

# 8.2 Target Generation

In [ ]:
# ga paremeters
POP_SELECT= .2
POP_CROSS = .9
POP_MUT = .1
POP_ELITE =.1

# function to be compared
crossover_funcs = [cx]
c_names =['CX']
mutation_funcs= [rsm]

# run parameters
pop_size = 230
target_generation = 50


# run and plot 

for w in  range(20):
    for i,cross in enumerate(crossover_funcs):   
       
        for mut in mutation_funcs:
            # Parallel
            jobs =[Job(pop_size,
                   POP_SELECT,
                   POP_CROSS,
                   POP_MUT,
                   POP_ELITE,
                   cross,
                   mut,
                   DIST_MAT) for _ in range(repeats)]

            jobs_RDD = sc.parallelize(jobs)
            jobs_RDD = jobs_RDD.map(lambda j: j.generate_population())
            jobs_RDD = jobs_RDD.map(lambda j: j.run(target_generation))
            jobs_res = jobs_RDD.map(lambda j: j.get_history()).collect()
            jobs_res = np.array(jobs_res).mean(axis=0)
          
    target_generation+=(w*5)

# 8.3 Selection

In [ ]:
t0 = time()

# ga paremeters
POP_SELECT= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
POP_CROSS = .9
POP_MUT = .1
POP_ELITE =.1


# function to be compared
crossover_funcs = [cx]
c_names =['CX']
mutation_funcs= [rsm]

# run parameters
pop_size = 230
target_generation = 905

pop_select_result = []

# run and plot 
for w in POP_SELECT: 
    for i,cross in enumerate(crossover_funcs):   
        for mut in mutation_funcs:
            # Parallel
            jobs =[Job(pop_size,
                   w,
                   POP_CROSS,
                   POP_MUT,
                   POP_ELITE,
                   cross,
                   mut,
                   DIST_MAT) for _ in range(repeats)]
            jobs_RDD = sc.parallelize(jobs)
            jobs_RDD = jobs_RDD.map(lambda j: j.generate_population())
            jobs_RDD = jobs_RDD.map(lambda j: j.run(target_generation))
            jobs_res = jobs_RDD.map(lambda j: j.get_history()).collect()
            jobs_res = np.array(jobs_res).mean(axis=0)
            pop_select_result.append(jobs_res)
       

t_min = (time() - t0) //60
print(f'Runtime: {t_min} minutes')

In [ ]:
best_fitness = [min(fitness) for fitness in pop_select_result]
plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] ,best_fitness )
plt.show()

# 8.4 Crossover

In [ ]:
t0 = time()

# ga paremeters
POP_SELECT= 0.4
POP_CROSS = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
POP_MUT = .1
POP_ELITE =.1


# function to be compared
crossover_funcs = [cx]
c_names =['CX']
mutation_funcs= [rsm]

# run parameters
pop_size = 230
target_generation = 905

pop_cros_result = []
# run and plot 
for w in POP_CROSS: 
    for i,cross in enumerate(crossover_funcs):   
        for mut in mutation_funcs:
            # Parallel
            jobs =[Job(pop_size,
                   POP_SELECT,
                   w,
                   POP_MUT,
                   POP_ELITE,
                   cross,
                   mut,
                   DIST_MAT) for _ in range(repeats)]
            jobs_RDD = sc.parallelize(jobs)
            jobs_RDD = jobs_RDD.map(lambda j: j.generate_population())
            jobs_RDD = jobs_RDD.map(lambda j: j.run(target_generation))
            jobs_res = jobs_RDD.map(lambda j: j.get_history()).collect()
            jobs_res = np.array(jobs_res).mean(axis=0)
            pop_cros_result.append(jobs_res)
            

t_min = (time() - t0) //60
print(f'Runtime: {t_min} minutes')

In [ ]:
best_fitness = [min(fitness) for fitness in pop_cros_result]
plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] ,best_fitness )
plt.show()

# 8.5 Mutation

In [ ]:
t0 = time()

# ga paremeters
POP_SELECT= 0.4
POP_CROSS = 0.4
POP_MUT = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9]
POP_ELITE =.1


# function to be compared
crossover_funcs = [cx]
c_names =['CX']
mutation_funcs= [rsm]

# run parameters
pop_size = 230
target_generation = 905

pop_mut_result= []

# run and plot 
for w in POP_MUT: 
    for i,cross in enumerate(crossover_funcs):   
        for mut in mutation_funcs:
            # Parallel
            jobs =[Job(pop_size,
                   POP_SELECT,
                   POP_CROSS,
                   w,
                   POP_ELITE,
                   cross,
                   mut,
                   DIST_MAT) for _ in range(repeats)]
            jobs_RDD = sc.parallelize(jobs)
            jobs_RDD = jobs_RDD.map(lambda j: j.generate_population())
            jobs_RDD = jobs_RDD.map(lambda j: j.run(target_generation))
            jobs_res = jobs_RDD.map(lambda j: j.get_history()).collect()
            jobs_res = np.array(jobs_res).mean(axis=0)
            pop_mut_result.append(jobs_res)
            

In [ ]:
best_fitness = [min(fitness) for fitness in pop_mut_result]
plt.plot([0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9] ,best_fitness )
plt.show()

# 9. Run with optimized GA parameters

In [36]:
from time import time

%run Functions.py
%run Population.py
%run Job.py

# load cities data from file
DIST_MAT = np.loadtxt('att48_d.txt')

# ga paremeters
POP_SELECT = 0.4
POP_CROSS = 0.4
POP_MUT = 0.15
POP_ELITE =0.1

# run parameters
pop_size = 230
target_generation = 905

# function to be compared
cross = cx
mut = rsm

# run
t0 = time()

job = Job(pop_size,POP_SELECT,POP_CROSS,POP_MUT,POP_ELITE,
          cross,mut,DIST_MAT)

job.generate_population()
job.run(target_generation)
path, length = job.get_optimum()
    
t_min = (time() - t0)
print(f'Runtime: {t_min} seconds')

print(f'Optimal Path:\n {path}')
print(f'Length: {length}')

Runtime: 47.30278944969177 seconds
Optimal Path:
 [ 7 18 44 31 38  9  8  1 22 16 41 34 29  2 26  4 35 45 10 24 42  5 48 39
 32 21 13 25 14 23  3 40 15 46 33 12 11 47 20 36 30 43 17 27 37 19  6 28]
Length: 34494.0
